# Importing Libraries, Loading Data and Other Good Stuff

In [ ]:
%load_ext nb_black
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from geopandas.plotting import _PolygonPatch
from shapely.affinity import scale

In [ ]:
econ_survey = pd.read_csv("economic_survey_2021.csv")

In [ ]:
econ_survey.head()

In [ ]:
econ_survey = econ_survey[["Name", "GDDP_Current"]]

In [ ]:
districts = gpd.read_file("District.shp")

In [ ]:
districts.head()

In [ ]:
districts = districts.to_crs("4326")

In [ ]:
econ_survey = econ_survey.rename({"Name": "KGISDist_1"}, axis=1)
econ_survey.head()

In [ ]:
districts = districts.merge(econ_survey)

# Plotting Shapefiles

In [ ]:
districts.plot()

In [ ]:
districts.plot(figsize=(20, 20))

# Choropleth Maps

In [ ]:
districts.plot(figsize=(20, 20), column="GDDP_Current")

In [ ]:
districts.plot(
    figsize=(20, 20), column="GDDP_Current", colormap="RdPu", scheme="percentiles"
)

In [ ]:
# Basic figure settings
fig, ax = plt.subplots(figsize=(20, 20), facecolor="white")

# Adding Text
ax.text(
    0.67,
    0.8,
    "KARNATAKA DISTRICT WISE GDP (In Lakh Rupees)",
    fontname="Sawasdee",
    fontsize=25,
    fontweight=3,
    transform=fig.transFigure,
)

# Turning off axes
ax.axis("off")

# Plotting district boundaries/outline
ax = districts.boundary.plot(color="black", linewidth=0.5, ax=ax)

# Plotting GDDP Data as Choropleth
districts.plot(
    ax=ax,
    column="GDDP_Current",
    cmap="YlOrRd",
    scheme="percentiles",
    classification_kwds={"pct": [10, 20, 30, 40, 50, 60, 70, 80, 99, 100]},
    legend=True,
    legend_kwds={
        "labels": [
            "12.8 to 18.7",
            "18.7 to 22.8",
            "22.8 to 25.7",
            "25.7 to 27.8",
            "27.8 to 31.0",
            "31.0 to 38.8",
            "38.8 to 39.7",
            "39.7 to 52.3",
            "52.3 to 445.11",
            "445.11 to 596.26",
        ],
        "loc": "center right",
        "bbox_to_anchor": (1.3, 0.5),
        "prop": {"family": "Sawasdee", "weight": 3, "size": 22},
    },
)

# Bubble Plots

In [ ]:
# Convert Polygon to a Point
centroids = districts.copy()
centroids["geometry"] = districts.centroid

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax = districts.boundary.plot(ax=ax)
centroids.plot(ax=ax, color="black")

In [ ]:
# Makes sure the point is inside the polygon
rep_points = districts.copy()
rep_points["geometry"] = districts.representative_point()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax = districts.boundary.plot(ax=ax)
rep_points.plot(ax=ax, color="black")

In [ ]:
# Scale these points to fit inside the map
rep_points["size"] = rep_points["GDDP_Current"] / 30000  # 30000 is by trail and error

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), facecolor="white")

ax.text(
    0.67,
    0.8,
    "KARNATAKA DISTRICT WISE GDP (In Lakh Rupees)",
    fontname="Sawasdee",
    fontsize=25,
    fontweight=3,
    transform=fig.transFigure,
)

ax.axis("off")

ax = districts.boundary.plot(color="black", linewidth=0.5, ax=ax)

rep_points.plot(
    ax=ax,
    column="GDDP_Current",
    markersize="size",  # For Bubble Plot
    cmap="YlOrRd",
    edgecolor="black",
    scheme="percentiles",
    classification_kwds={"pct": [10, 20, 30, 40, 50, 60, 70, 80, 99, 100]},
    legend=True,
    legend_kwds={
        "labels": [
            "12.8 to 18.7",
            "18.7 to 22.8",
            "22.8 to 25.7",
            "25.7 to 27.8",
            "27.8 to 31.0",
            "31.0 to 38.8",
            "38.8 to 39.7",
            "39.7 to 52.3",
            "52.3 to 445.11",
            "445.11 to 596.26",
        ],
        "loc": "center right",
        "bbox_to_anchor": (1.3, 0.5),
        "prop": {"family": "Sawasdee", "weight": 3, "size": 22},
    },
)

# Cartogram

In [ ]:
cartogram = districts.copy()
cartogram["scale_factor"] = cartogram["GDDP_Current"] / cartogram["GDDP_Current"].max()
cartogram["repr_point"] = cartogram.representative_point()

In [ ]:
# Scale each geometry
scaled_geoms = [
    scale(
        x["geometry"],
        xfact=x["scale_factor"],
        yfact=x["scale_factor"],
        origin=x["repr_point"],
    )
    for x in cartogram.to_dict(orient="records")
]

cartogram["geometry"] = scaled_geoms

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), facecolor="white")

ax.text(
    0.67,
    0.8,
    "KARNATAKA DISTRICT WISE GDP (In Lakh Rupees)",
    fontname="Sawasdee",
    fontsize=25,
    fontweight=3,
    transform=fig.transFigure,
)


ax.axis("off")

ax = districts.boundary.plot(color="black", linewidth=0.5, ax=ax)

cartogram.plot(
    ax=ax,
    column="GDDP_Current",
    cmap="YlOrRd",
    edgecolor="black",
    linewidth=0.2,
    scheme="percentiles",
    classification_kwds={"pct": [10, 20, 30, 40, 50, 60, 70, 80, 99, 100]},
    legend=True,
    legend_kwds={
        "labels": [
            "12.8 to 18.7",
            "18.7 to 22.8",
            "22.8 to 25.7",
            "25.7 to 27.8",
            "27.8 to 31.0",
            "31.0 to 38.8",
            "38.8 to 39.7",
            "39.7 to 52.3",
            "52.3 to 445.11",
            "445.11 to 596.26",
        ],
        "loc": "center right",
        "bbox_to_anchor": (1.3, 0.5),
        "prop": {"family": "Sawasdee", "weight": 3, "size": 22},
    },
)

# Point Plots Revisited, Briefly

In [ ]:
assemblies = gpd.read_file("AC_Boundary.shp")
assemblies = assemblies.to_crs("4326")

In [ ]:
assemblies.plot(figsize=(10, 10))

In [ ]:
assemblies["geometry"] = assemblies.representative_point()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), facecolor="white")

ax.text(
    0.67,
    0.8,
    "KARNATAKA LOKSABHA ASSEMBLIES",
    fontname="Sawasdee",
    fontsize=25,
    fontweight=3,
    transform=fig.transFigure,
)

ax.axis("off")

ax = districts.boundary.plot(color="black", linewidth=0.5, ax=ax)

assemblies.plot(ax=ax, color="#ffb536", edgecolor="black")

# Density Plots on a Map

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), facecolor="white")

ax.text(
    0.67,
    0.8,
    "KARNATAKA LOKSABHA ASSEMBLIES",
    fontname="Sawasdee",
    fontsize=25,
    fontweight=3,
    transform=fig.transFigure,
)

ax.axis("off")

ax = districts.boundary.plot(color="black", linewidth=0.5, ax=ax)

# Plot Seaborn kdeplot with x and y values from geometry
kde = sns.kdeplot(
    x=assemblies["geometry"].x, y=assemblies["geometry"].y, fill=True, cmap="Blues"
)

# Clipping the heatmap, not so straight forward

In [ ]:
karnataka = districts.copy()
karnataka["dissolve"] = "Dissolve"
karnataka = karnataka.dissolve(by="dissolve")
karnataka = karnataka.geometry[0]

not_karnataka = karnataka.envelope
not_karnataka = scale(not_karnataka.envelope, xfact=1.2, yfact=1.2)
not_karnataka = not_karnataka.difference(karnataka)

In [ ]:
str(type(not_karnataka))

In [ ]:
patches = [
    _PolygonPatch(geom, facecolor="white", edgecolor="white")
    for geom in not_karnataka.geoms
]

fig, ax = plt.subplots(figsize=(20, 20))

ax.text(
    0.67,
    0.8,
    "KARNATAKA LOKSABHA ASSEMBLIES",
    fontname="Sawasdee",
    fontsize=25,
    fontweight=3,
    transform=fig.transFigure,
)

ax.axis("off")

ax = districts.boundary.plot(color="black", linewidth=0.5, ax=ax)

kde = sns.kdeplot(
    ax=ax,
    x=assemblies["geometry"].x,
    y=assemblies["geometry"].y,
    fill=True,
    cmap="Blues",
)

for patch in patches:
    ax.add_patch(patch)

# Geospatial Heatmaps

In [ ]:
karnataka.bounds

In [ ]:
grid_space = 0.01

grid_lon = np.arange(74.0855, 78.5878 + grid_space, grid_space)
grid_lat = np.arange(11.5949, 18.4777 + grid_space, grid_space)

all_lats = np.meshgrid(grid_lon, grid_lat)[1].ravel()
all_lons = np.meshgrid(grid_lon, grid_lat)[0].ravel()
del grid_lat, grid_lon

pairs = list(zip(all_lats, all_lons))
del all_lats, all_lons

grid = pd.DataFrame(pairs, columns=["lat", "lon"])

In [ ]:
grid

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
rep_points

In [ ]:
train = pd.DataFrame()
train["lat"] = rep_points["geometry"].y
train["lon"] = rep_points["geometry"].x
train["GDDP_Current"] = rep_points["GDDP_Current"]

In [ ]:
model = KNeighborsRegressor(weights="distance", n_neighbors=len(train) - 1).fit(
    train[["lat", "lon"]], train["GDDP_Current"].to_numpy().reshape(-1, 1)
)

In [ ]:
grid["GDDP_Current"] = model.predict(grid[["lat", "lon"]])

In [ ]:
grid = gpd.GeoDataFrame(
    grid, geometry=gpd.points_from_xy(grid["lon"], grid["lat"]), crs="4326"
)

In [ ]:
patches = [
    _PolygonPatch(geom, facecolor="white", edgecolor="white")
    for geom in not_karnataka.geoms
]

fig, ax = plt.subplots(figsize=(20, 20), facecolor="white")

ax.set_facecolor("white")

ax.text(
    0.67,
    0.8,
    "KARNATAKA DISTRICT WISE GDP (In Lakh Rupees)",
    fontname="Sawasdee",
    fontsize=25,
    fontweight=3,
    transform=fig.transFigure,
)

ax.axis("off")

ax = districts.boundary.plot(color="black", linewidth=0.5, ax=ax)
grid.plot(
    ax=ax,
    column="GDDP_Current",
    cmap="YlOrRd",
    linewidth=0.2,
    scheme="percentiles",
    classification_kwds={"pct": [10, 20, 30, 40, 50, 60, 70, 80, 99, 100]},
    legend=True,
    legend_kwds={
        "labels": [
            "12.8 to 18.7",
            "18.7 to 22.8",
            "22.8 to 25.7",
            "25.7 to 27.8",
            "27.8 to 31.0",
            "31.0 to 38.8",
            "38.8 to 39.7",
            "39.7 to 52.3",
            "52.3 to 445.11",
            "445.11 to 596.26",
        ],
        "loc": "center right",
        "bbox_to_anchor": (1.3, 0.5),
        "prop": {"family": "Sawasdee", "weight": 3, "size": 22},
    },
)
for patch in patches:
    ax.add_patch(patch)

# Interactive Plotting with Geopandas

In [ ]:
districts.explore(
    column="GDDP_Current",
    cmap="YlOrRd",
    scheme="percentiles",
    k=10,
    classification_kwds={"pct": [10, 20, 30, 40, 50, 60, 70, 80, 99, 100]},
    legend=True,
    legend_kwds={
        "colorbar": False,
        "labels": [
            "12.8 to 18.7",
            "18.7 to 22.8",
            "22.8 to 25.7",
            "25.7 to 27.8",
            "27.8 to 31.0",
            "31.0 to 38.8",
            "38.8 to 39.7",
            "39.7 to 52.3",
            "52.3 to 445.11",
            "445.11 to 596.26",
        ],
    },
)

In [ ]:
districts.explore(
    column="GDDP_Current",
    cmap="YlOrRd",
    scheme="percentiles",
    k=10,
    classification_kwds={"pct": [10, 20, 30, 40, 50, 60, 70, 80, 99, 100]},
    tiles=None,
    legend=True,
    legend_kwds={
        "colorbar": False,
        "labels": [
            "12.8 to 18.7",
            "18.7 to 22.8",
            "22.8 to 25.7",
            "25.7 to 27.8",
            "27.8 to 31.0",
            "31.0 to 38.8",
            "38.8 to 39.7",
            "39.7 to 52.3",
            "52.3 to 445.11",
            "445.11 to 596.26",
        ],
    },
)

# Data Soruces
 * Govt GIS Portals
 * ArcGIS Open Data Hub - https://hub.arcgis.com/
 * Natural Earth - https://www.naturalearthdata.com
 * OpenStreetMap - https://overpass-turbo.eu/ 

# gspatial_plot

* Open Source geospatial plotting library
* seaborn equivalent for geospatial plots
* simplified way to plot popular type of thematic maps
* better default settings
* customization parameters
* built on top of geopandas, cross compatible with geopandas and matplotlib

- Star it on Github - https://github.com/ambeelabs/gspatial_plot
- Install it PyPi - https://pypi.org/project/gspatial-plot/ (pip install gspatial-plot)
- Readthedocs - https://gspatial-plot.readthedocs.io (Examples and more explanations will be added)
- In the plans - Blog series sharing tutorials and recipes

Note: The library is in alpha and there might be some bugs that needs fixing, please report bugs if you find any.
